#### Import packages

In [1]:
# import torch.multiprocessing as mp
# mp.set_start_method('spawn') 
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import math
import shutil
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
import random
import cv2
import copy

from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.layers import UpSampling2D, BatchNormalization, Flatten, Dense, GlobalMaxPooling2D, Input, concatenate
from keras.layers.core import Dropout
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.optimizers import SGD

seed = 0
np.random.seed(seed)
from utils_diff_ch_scaled import *

Using TensorFlow backend.
/home/arsh/anaconda3/envs/deepLearn/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


#### Load and Prepare Data

In [2]:
# def scale_range (input_, max_, min_):
#     input_[:,0,:,:] += -(-45.594448)
#     input_[:,0,:,:] /= 82.68497099999999 / (max_ - min_)
#     input_[:,0,:,:] += min_
#     input_[:,1,:,:] += -(-49.083500000000001)
#     input_[:,1,:,:] /= 69.686413000000002 / (max_ - min_)
#     input_[:,1,:,:] += min_
#     return input_

def scale_range (input_, max_, min_):
    return input_

In [3]:
data = pd.read_json('./Data/train.json', dtype={'inc_angle': str})
test = pd.read_json('./Data/test.json', dtype={'inc_angle': str})

In [4]:
data['band_1'] = data['band_1'].apply(lambda x : np.array(x).reshape(75, 75))
data['band_2'] = data['band_2'].apply(lambda x : np.array(x).reshape(75, 75))

test['band_1'] = test['band_1'].apply(lambda x : np.array(x).reshape(75, 75))
test['band_2'] = test['band_2'].apply(lambda x : np.array(x).reshape(75, 75))

test['not_machine_generated'] = test['inc_angle'].apply(lambda x: len(str(x).split('.')[1])) <= 4
data['inc_angle'] = pd.to_numeric(data['inc_angle'], errors='coerce')
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')

In [5]:
# Impute missing incidence angles
data_impute = data.dropna(axis=0, how='any')
data_impute_b1 = np.array([im.reshape(75*75) for im in data_impute.loc[:,'band_1']])
data_impute_b2 = np.array([im.reshape(75*75) for im in data_impute.loc[:,'band_2']])
data_impute_X = np.hstack(np.array((data_impute_b1, data_impute_b2)))
data_impute_y = np.array(data_impute[['inc_angle']])

data_impute_prec = data[~data.index.isin(data_impute.index.values)]
data_impute_b1_prec = np.array([im.reshape(75*75) for im in data_impute_prec.loc[:,'band_1']])
data_impute_b2_prec = np.array([im.reshape(75*75) for im in data_impute_prec.loc[:,'band_2']])
data_impute_X_prec = np.hstack(np.array((data_impute_b1_prec, data_impute_b2_prec)))

modelImpute = LinearRegression()
modelImpute = modelImpute.fit(data_impute_X, data_impute_y)
data_impute_prec['inc_angle'] = modelImpute.predict(data_impute_X_prec)

data.loc[data_impute_prec.index] = data_impute_prec

/home/arsh/anaconda3/envs/deepLearn/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
ids_train = np.array(data.index)
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

#### Keras Model

In [9]:
# Hyper Parameters
width = 75
height = 75
channels = 3

In [26]:
def train_generator():
    while True:
        for start in range(0, len(ids_train_split), batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, len(ids_train_split))
            ids_train_batch = ids_train_split[start:end]
            for idx in ids_train_batch:
                
                train = data.iloc[idx,]
                band_1_tr = np.concatenate([im for im in train['band_1']]).reshape(-1, width, height)
                band_2_tr = np.concatenate([im for im in train['band_2']]).reshape(-1, width, height)
                X_train = np.concatenate((band_1_tr, band_2_tr), axis=0)

                im = X_train
                im = randomErodeDilate(im, u=0.5)
                im = randomZoomOut(im, u=0.5)
                im = randomNoisy(im, u=0.5)
                im = randomShift(im, u=0.5)
                
                # Mean Substraction
                im[0, :, :] -= -45.594448
                im[1, :, :] -= -49.083500000000001
                band_3 = (im[0, :, :]+im[1, :, :])/2
                im = np.concatenate((im, band_3.reshape((1, width, height))), axis=0)
                
                x_batch.append(im.reshape((75,75,channels)))
                y_batch.append(train['is_iceberg'].astype(np.float32))
            
            x_batch = np.array(x_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)
            yield x_batch, y_batch


def valid_generator():
    while True:
        for start in range(0, len(ids_valid_split), batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for idx in ids_valid_batch:
                
                val = data.iloc[idx,]
                band_1_val = np.concatenate([im for im in val['band_1']]).reshape(-1, width, height)
                band_2_val = np.concatenate([im for im in val['band_2']]).reshape(-1, width, height)
                X_val = np.concatenate((band_1_val, band_2_val), axis=0)

                im = X_val
                
                # Mean Substraction
                im[0, :, :] -= -45.594448
                im[1, :, :] -= -49.083500000000001
                band_3 = (im[0, :, :]+im[1, :, :])/2
                im = np.concatenate((im, band_3.reshape((1, width, height))), axis=0)
                
                x_batch.append(im.reshape((75,75,channels)))
                y_batch.append(val['is_iceberg'].astype(np.float32))
            
            x_batch = np.array(x_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)
            yield x_batch, y_batch

In [27]:
def getVggModel():
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=(width, height, channels), classes=1)
    x = base_model.get_layer('block5_pool').output

    x = GlobalMaxPooling2D()(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu', name='fc3')(x)
    x = Dropout(0.5)(x)
    
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model
model = getVggModel()

In [7]:
def train_generator(ids_train_split=ids_train_split, batch_size=32):
    while True:
        for start in range(0, len(ids_train_split), batch_size):
            x_batch_0 = []
            x_batch_1 = []
            y_batch = []
            end = min(start + batch_size, len(ids_train_split))
            ids_train_batch = ids_train_split[start:end]
            for idx in ids_train_batch:
                
                train = data.iloc[idx,]
                band_1_tr = np.concatenate([im for im in train['band_1']]).reshape(-1, width, height)
                band_2_tr = np.concatenate([im for im in train['band_2']]).reshape(-1, width, height)
                X_train = np.concatenate((band_1_tr, band_2_tr), axis=0)
                X_train = [X_train, np.array(train['inc_angle'])]

                im = X_train[0]
                im = randomErodeDilate(im, u=0.5)
                im = randomZoomOut(im, u=0.5)
                im = randomNoisy(im, u=0.5)
                im = randomShift(im, u=0.5)
                
                # Mean Substraction
                im[0, :, :] -= -45.594448
                im[1, :, :] -= -49.083500000000001
                band_3 = (im[0, :, :]+im[1, :, :])/2
                im = np.concatenate((im, band_3.reshape((1, width, height))), axis=0)
                
                x_batch_0.append(im.reshape((75,75,channels)))
                x_batch_1.append(X_train[1].astype(np.float32))
                y_batch.append(train['is_iceberg'].astype(np.float32))
            
            x_batch = [np.array(x_batch_0, np.float32), np.array(x_batch_1, np.float32)]
            y_batch = np.array(y_batch, np.float32)
            yield x_batch, y_batch


def valid_generator(ids_valid_split=ids_valid_split, batch_size=32):
    while True:
        for start in range(0, len(ids_valid_split), batch_size):
            x_batch_0 = []
            x_batch_1 = []
            y_batch = []
            end = min(start + batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for idx in ids_valid_batch:
                
                val = data.iloc[idx,]
                band_1_val = np.concatenate([im for im in val['band_1']]).reshape(-1, width, height)
                band_2_val = np.concatenate([im for im in val['band_2']]).reshape(-1, width, height)
                X_val = np.concatenate((band_1_val, band_2_val), axis=0)
                X_val = [X_val, np.array(val['inc_angle'])]

                im = X_val[0]
                
                # Mean Substraction
                im[0, :, :] -= -45.594448
                im[1, :, :] -= -49.083500000000001
                band_3 = (im[0, :, :]+im[1, :, :])/2
                im = np.concatenate((im, band_3.reshape((1, width, height))), axis=0)
                
                x_batch_0.append(im.reshape((75,75,channels)))
                x_batch_1.append(X_val[1].astype(np.float32))
                y_batch.append(val['is_iceberg'].astype(np.float32))
            
            x_batch = [np.array(x_batch_0, np.float32), np.array(x_batch_1, np.float32)]
            y_batch = np.array(y_batch, np.float32)
            yield x_batch, y_batch

In [13]:
def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=(width, height, channels), classes=1)
    x = base_model.get_layer('block5_pool').output
    
    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(inputs=[base_model.input, input_2], outputs=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model
model = getVggAngleModel()

In [10]:
epochs = 300
batch_size = 32

callbacks = [EarlyStopping(monitor='val_loss',
                           patience=100,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=60,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='Keras_models/best_weights.hdf5',
                             save_best_only=True,
                             save_weights_only=True),
             TensorBoard(log_dir='keras_logs')]

model.fit_generator(generator=train_generator(),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(batch_size)),
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator(),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(batch_size)))

Epoch 1/300
41/41 [==============================] - 6s - loss: 0.8505 - acc: 0.5596 - val_loss: 0.5582 - val_acc: 0.7165
Epoch 2/300
41/41 [==============================] - 5s - loss: 0.6310 - acc: 0.6324 - val_loss: 0.5268 - val_acc: 0.7103
Epoch 3/300
41/41 [==============================] - 4s - loss: 0.6700 - acc: 0.6042 - val_loss: 0.5468 - val_acc: 0.7040
Epoch 4/300
41/41 [==============================] - 5s - loss: 0.5537 - acc: 0.7061 - val_loss: 0.4385 - val_acc: 0.7664
Epoch 5/300
41/41 [==============================] - 5s - loss: 0.5141 - acc: 0.7160 - val_loss: 0.4422 - val_acc: 0.7882
Epoch 6/300
41/41 [==============================] - 5s - loss: 0.4688 - acc: 0.7556 - val_loss: 0.3881 - val_acc: 0.8100
Epoch 7/300
41/41 [==============================] - 5s - loss: 0.4260 - acc: 0.7747 - val_loss: 0.4539 - val_acc: 0.7321
Epoch 8/300
41/41 [==============================] - 5s - loss: 0.4268 - acc: 0.7667 - val_loss: 0.4600 - val_acc: 0.7383
Epoch 9/300
41/41 [=====

#### KFold

In [10]:
ids_train = np.array(data.index)
band_1_test = np.concatenate([im for im in test['band_1']]).reshape(-1, width, height)
band_2_test = np.concatenate([im for im in test['band_2']]).reshape(-1, width, height)
im = np.stack((band_1_test, band_2_test), axis=3)
# Mean Substraction
im[:, :, :, 0] -= -45.594448
im[:, :, :, 1] -= -49.083500000000001
band_3 = (im[:, :, :, 0]+im[:, :, :, 1])/2
im = np.concatenate((im, band_3.reshape((-1, width, height, 1))), axis=3)
X_test = [im.reshape((len(im), width, height, channels)), np.array(test['inc_angle'])]

In [11]:
num_epochs = 500
batch_size = 64

In [16]:
kfold = 5
kfold_scores = []

results = []
fold_predictions = []
sss = KFold(n_splits=kfold, random_state=0)
for i, (train_index, test_index) in enumerate(sss.split(ids_train)):
    
    model = getVggAngleModel()
    print('Fold [%d/%d]' % (i+1, kfold))

    # Train
    callbacks = [EarlyStopping(monitor='val_loss',
                           patience=100,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=60,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='./Models/v1Nov_v1/model_fold_'+str(i+1)+'.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]
    
    model.fit_generator(generator=train_generator(train_index, batch_size),
                    steps_per_epoch=np.ceil(float(len(train_index)) / float(batch_size)),
                    epochs=num_epochs,
                    verbose=0,
                    callbacks=callbacks,
                    validation_data=valid_generator(test_index, batch_size),
                    validation_steps=np.ceil(float(len(test_index)) / float(batch_size)))
    model.load_weights('./Models/v1Nov_v1/model_fold_'+str(i+1)+'.hdf5')
    
    best_fold_predictions = model.predict_generator(valid_generator(test_index, 1), 
                                                    np.ceil(float(len(ids_valid_split)) / float(1)))[:,0]
    best_prec1 = model.evaluate_generator(valid_generator(test_index, 1), 
                                                    np.ceil(float(len(ids_valid_split)) / float(1)))[0]
        
    print('Val Score : %f' % (best_prec1))
    fold_predictions.append(best_fold_predictions)
    kfold_scores.append(best_prec1)
    
    # Predict
    results.append(np.array(model.predict(X_test)))

Fold [1/5]

Epoch 00223: reducing learning rate to 0.00010000000474974513.
Epoch 00263: early stopping
Val Score : 0.294870
Fold [2/5]

Epoch 00254: reducing learning rate to 0.00010000000474974513.

Epoch 00326: reducing learning rate to 1.0000000474974514e-05.
Epoch 00366: early stopping
Val Score : 0.246940
Fold [3/5]

Epoch 00227: reducing learning rate to 0.00010000000474974513.
Epoch 00267: early stopping
Val Score : 0.322410
Fold [4/5]

Epoch 00209: reducing learning rate to 0.00010000000474974513.
Epoch 00249: early stopping
Val Score : 0.280067
Fold [5/5]

Epoch 00235: reducing learning rate to 0.00010000000474974513.
Epoch 00275: early stopping
Val Score : 0.231487


In [17]:
np.array(kfold_scores).mean()

0.27515492565796551

In [18]:
np.array(kfold_scores).std()

0.032707387587890796

In [27]:
np.array(results).mean(axis=0).shape

(8424, 1)

In [19]:
sub_num = 56
os.makedirs("./Models/Sub "+str(sub_num))
np.savetxt("./Models/Sub "+str(sub_num)+"/results.csv", np.array(results), delimiter=",")
np.savetxt("./Models/Sub "+str(sub_num)+"/cv_fold_results.csv", np.concatenate(fold_predictions), delimiter=",")
sub = pd.read_csv('./Data/sample_submission.csv')
sub['is_iceberg'] = np.array(results).mean(axis=0)
sub.to_csv('./Submissions/Sub '+str(sub_num)+' - 5-fold _ Val-2751.csv', index=False)

In [3]:
sub = pd.read_csv('./Data/sample_submission.csv')

In [4]:
sub1 = pd.read_csv('./Submissions/Sub 46 - 5-fold _ Val-1563.csv')['is_iceberg']
sub2 = pd.read_csv('./Submissions/Sub 47 - 5-fold _ Val-1679.csv')['is_iceberg']
sub3 = pd.read_csv('./Submissions/Sub 48 - 5-fold _ Val-1583.csv')['is_iceberg']
sub4 = pd.read_csv('./Submissions/Sub 22 - 5-fold _ Val-1571.csv')['is_iceberg']
# sub5 = pd.read_csv('./Submissions/Sub 8 - 5-fold _ Val-1479.csv')['is_iceberg']
# sub6 = pd.read_csv('./Submissions/Sub 16 - 5-fold _ Val-1422.csv')['is_iceberg']

In [5]:
results = (np.array(sub1)*np.array(sub2)*np.array(sub3)*np.array(sub4))**(1/4)
sub['is_iceberg'] = np.array(results)
sub.to_csv('./Submissions/Sub 49 - Ensemble_22_46_47_48 - Geometric Mean.csv', index=False)

#### Error Analysis

In [24]:
def accuracy(loader):
    iceNet.eval()
    correct = 0
    total = 0
    loss = []
    inc_features = []
    inc_feature_angles = []
    inc_labels = []
    for features, features_angle, labels in loader:
        features = Variable(features, volatile=True).cuda()
        features_angle = Variable(features_angle, volatile=True).cuda()
        labels = Variable(labels, volatile=True).cuda()
        outputs = iceNet(features, features_angle)
        _loss = criterion(outputs, labels)
        loss.append(_loss)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.data).sum()
        inc_features.append(features.data.cpu().numpy()[np.where((predicted!=labels.data).cpu().numpy())])
        inc_feature_angles.append(features_angle.data.cpu().numpy()[np.where((predicted!=labels.data).cpu().numpy())])
        inc_labels.append(labels.data.cpu().numpy()[np.where((predicted!=labels.data).cpu().numpy())])
        
    return np.mean(loss).data[0], (100 * correct / total), [inc_features, inc_feature_angles, inc_labels]

In [25]:
kfold = 5
kfold_scores = []

results = []
sss = KFold(n_splits=kfold, random_state=0)
for i, (train_index, test_index) in enumerate(sss.split(X_KF[0], X_KF[1], y_KF)):
    if i == 2:
        X_train_KF, X_valid_KF = [X_KF[0][train_index], X_KF[1][train_index]], [X_KF[0][test_index], X_KF[1][test_index]]
        y_train_KF, y_valid_KF = y_KF[train_index], y_KF[test_index]

        # Define model
        iceNet = net().apply(weight_init).cuda()

        # Loss and Optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(iceNet.parameters(), lr=learning_rate, weight_decay=0.0001)

        # Data Loader
        train_dataset_KF = icebergDataset(X_train_KF, y_train_KF, transform=True)
        val_dataset_KF = icebergDataset(X_valid_KF, y_valid_KF)

        train_loader_KF = torch.utils.data.DataLoader(dataset=train_dataset_KF, batch_size=batch_size, shuffle=True)
        val_loader_KF = torch.utils.data.DataLoader(dataset=val_dataset_KF, batch_size=batch_size, shuffle=False)

        print('Fold [%d/%d]' % (i+1, kfold))
        # Train
        best_prec1 = 1
        for epoch in range(num_epochs):
            epoch_train_loss = []
            for idx, (features, features_angle, labels) in enumerate(train_loader_KF):
                iceNet.train()
                features = Variable(features).cuda()
                features_angle = Variable(features_angle).cuda()
                labels = Variable(labels).cuda()
                # Forward + Backward + Optimize
                optimizer.zero_grad()
                outputs = iceNet(features, features_angle)
                loss = criterion(outputs, labels)
                epoch_train_loss.append(loss)
                loss.backward()
                optimizer.step()
            
            acc = accuracy(val_loader_KF)
            prec1 = acc[0]

            # Save best model
            is_best = prec1 < best_prec1
            best_prec1 = min(prec1, best_prec1)
            if is_best:
                incorrect = acc[2]
print(best_prec1)

/home/arsh/anaconda3/envs/torch/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


Fold [3/5]
0.2119673639535904


In [26]:
incorrect[0] = np.vstack(incorrect[0])
incorrect[1] = np.vstack(incorrect[1])
incorrect[2] = np.hstack(incorrect[2])

In [46]:
def randomZoomOut(im, zoom_ratio=(-0.5, 0.5), u=0.5):
    if np.random.random() < u:
        height, width = im[0].shape
        
        zoom = height*np.random.uniform(zoom_ratio[0], zoom_ratio[1])
        if int(zoom)<0:
            zoom = abs(int(zoom))
            im_0 = cv2.resize(im[0, zoom//2:-zoom//2, zoom//2:-zoom//2], (height, width))
            im_1 = cv2.resize(im[1, zoom//2:-zoom//2, zoom//2:-zoom//2], (height, width))
#            im_2 = cv2.resize(im[2, zoom//2:-zoom//2, zoom//2:-zoom//2], (height, width))
#            im_3 = cv2.resize(im[3, zoom//2:-zoom//2, zoom//2:-zoom//2], (height, width))
            return np.stack((im_0,im_1))
        
        if zoom<0:
            zoom = 0
        im_0 = cv2.resize(cv2.copyMakeBorder(im[0],int(zoom//2),int(zoom//2),int(zoom//2),int(zoom//2),
                                   cv2.BORDER_REFLECT101), (height, width))
        im_1 = cv2.resize(cv2.copyMakeBorder(im[1],int(zoom//2),int(zoom//2),int(zoom//2),int(zoom//2),
                                   cv2.BORDER_WRAP), (height, width))
#        im_2 = cv2.resize(cv2.copyMakeBorder(im[2],int(zoom//2),int(zoom//2),int(zoom//2),int(zoom//2),
#                                   cv2.BORDER_REPLICATE), (height, width))
#        im_3 = cv2.resize(cv2.copyMakeBorder(im[3],int(zoom//2),int(zoom//2),int(zoom//2),int(zoom//2),
#                                   cv2.BORDER_REPLICATE), (height, width))
        return np.stack((im_0,im_1))
    
    return im

In [47]:
for idx, im in enumerate(incorrect[0]):
    f, axarr = plt.subplots(2, 2, figsize=(10,8))
    plt.title('Label = '+str(incorrect[2][idx])+', Inc Angle = '+str(incorrect[1][idx][0]))
    zm = randomZoomOut(im, (0, 0.5), 1)
    axarr[0, 0].imshow(im[0], cmap='gray')
    axarr[0, 1].imshow(im[1], cmap='gray')
    axarr[1, 0].imshow(zm[0], cmap='gray')
    axarr[1, 1].imshow(zm[1], cmap='gray')
    plt.show()

error: /home/travis/miniconda/conda-bld/conda_1486587069159/work/opencv-3.1.0/modules/core/src/copy.cpp:928: error: (-5) Unknown/unsupported border type in function borderInterpolate
